In [3]:
import numpy
import csv

   0.516219704459  1.60967137364  Unnamed: 2
0        0.850085      14.814006         NaN
1       -0.840629      -8.738649         NaN
2        0.227433      -0.274344         NaN
3       -0.649508      -8.683412         NaN
4        0.621241       5.992031         NaN
                   X   y
 0.194241   1.931291 NaN
 0.878633  17.545736 NaN
-0.276443  -1.173417 NaN
 0.996344  32.234868 NaN
 0.648838   8.063874 NaN
